In [4]:
import librosa  
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm.autonotebook import tqdm
import noisereduce as nr
from jupyterthemes import jtplot


C:\Users\franc\Anaconda3\envs\Py379\lib\site-packages\ipykernel_launcher.py:6: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  


In [5]:
jtplot.style(theme='monokai', context='notebook', ticks=True, gridlines='--')

In [6]:
wavFolder = os.path.join("bird_data","wav_files")
wavFiles = os.listdir(wavFolder)
test = os.path.join(wavFolder, wavFiles[0])

# Feature Engineering
Feature 1: MFCC with cepstral filtering <br>
Feature 2: Mel-scaled spectrogram


## Load Audio Signal Function

In [7]:
# Load signal and apply noise reduction if required
def loadSignal(file, downsample=True, denoise=False):
    SR = (22050 if downsample else 44100)
    signal, sr = librosa.load(file, sr=SR, mono=True)
    if denoise:
        rsignal = nr.reduce_noise(audio_clip=signal, noise_clip=signal, verbose=False)
        return rsignal, sr
    else:
        return signal, sr

## Spectrogram Functions

In [8]:
# Plot mel-spectrogram
def Mel_Spectrogram(FILE, N_FFT, HOP_LENGTH, WINDOW, FILTER=False):
    FMIN = (1400 if FILTER else 0)
    signal, sr = loadSignal(FILE, True, True)  
    mel = librosa.feature.melspectrogram(y=signal, sr=sr, n_fft=N_FFT, hop_length=HOP_LENGTH, htk=True, window=WINDOW, fmin=FMIN, fmax=sr/2) 
    S_dB = librosa.power_to_db(mel**2,ref=np.max)
    return S_dB

In [13]:
# Mel-frequency cepstral coefficients (MFCCs)
def MFCC(FILE, N_FFT, N_MFCC, HOP_LENGTH, DCT=2, FILTER=False):
    LIFTER=(2*N_MFCC if FILTER else 0) # cesptral filtering
    signal, sr = loadSignal(FILE, True, True)  
    mfccs = librosa.feature.mfcc(y=signal, sr=sr, n_fft = N_FFT, n_mfcc=N_MFCC, hop_length=HOP_LENGTH, htk=True, lifter=LIFTER, dct_type=DCT)
    S_dB = librosa.power_to_db(mfccs**2,ref=np.max)
    return S_dB